# Step-by-step guide to building MuSIC v1 map

Details for each section available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map).

## Installation
Details available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map#installation).

In [ ]:
%%bash
conda create -n music python=3.6.2 anaconda
source activate music
pip install -r ./installation/requirements.txt

## Input Data

We here provide the 1024-dimension embeddings for the 1,451 images and the 661 proteins used in MuSIC v1. 

Detailed documentation available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map#input-data).

In [1]:
import pandas as pd
IF_emd = pd.read_csv('./Examples/IF_image_embedding.csv', header=None)
APMS_emd = pd.read_csv('./Examples/APMS_embedding.MuSIC.csv', header=None)

In [2]:
IF_emd.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,IF_1,HLA-DPA1,0.022881,-0.060293,-0.140246,0.037648,0.009863,0.292151,-0.042231,0.020537,...,-0.036327,0.166945,0.023639,0.601420,-0.078425,0.192180,0.011603,0.031387,-0.029221,0.23038
1,IF_2,HLA-DPA1,0.022881,-0.113740,-0.140246,0.032609,0.191921,0.401565,-0.042231,0.027777,...,-0.129697,0.103786,0.023639,0.654706,-0.078425,0.156273,0.011603,0.031387,-0.001136,0.20808


In [3]:
APMS_emd.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
0,APMS_1,RRS1,0.075910,0.161315,-0.025731,0.071347,-0.175898,0.041408,-0.061304,-0.136247,...,-0.052199,-0.018137,0.042997,0.246699,-0.043538,0.016049,-0.147477,-0.049635,-0.001222,-0.172205
1,APMS_2,SNRNP70,-0.019872,0.083736,0.151332,0.080374,-0.053558,0.067913,-0.057474,-0.114813,...,-0.014049,-0.154981,-0.187242,-0.082924,-0.089952,-0.091610,-0.051024,-0.005062,-0.170704,0.042429


## Step 1: Generate gold-standard protein-protein proximity values
Detailed documentation available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map#step-1-generate-gold-standard-protein-protein-proximity-values).

In [4]:
%%bash
python calibrate_pairwise_distance.py \
--protein_file ./Examples/MuSIC_proteins.txt \
--outprefix ./Examples/output/test

Calibration Function:
log10(D) = 1.05 * log10(C) + -0.14
R2 = 0.89

Analyzing 661 proteins:
DMAP1, COLEC12, FBXL14, ASB13, CD72, EYA3, OAZ2, CCNDBP1, CDKN2B, PICK1, ZNF232, ZNF212, FKBPL, TRAPPC5, LMX1B, SUCLA2, DPYSL5, ORC2, TBKBP1, HSD17B6, RRN3, AGK, CUEDC2, KLHL15, UTP18, SEPT1, NAT14, PRDM16, KIDINS220, SCML1, DACH1, MCOLN2, PATZ1, TP73, ZNF224, EEF1D, WASF2, CREB5, WDR90, PRPSAP1, CASC3, ZDBF2, PHLPP2, MRPL20, SALL2, SMARCD2, CTU2, SFPQ, HUS1B, MTOR, RPL7L1, MAD2L2, SMAD9, GPR50, FHIT, RNF220, FAM160A2, ZNF416, SLC35F2, COL2A1, SDC2, GTF3C6, XKR8, RPL28, CASP8AP2, QPCT, FUZ, ZNF280B, NOS1AP, MCAT, SLC35F1, CTR9, LIN28B, SEPHS2, DNAJA3, CDO1, NASP, TAF11, ANKRD54, NSMCE4A, CLASRP, SYN2, WBP11, CAMK1, MORF4L1, TNNI3, CCDC130, TWIST2, ZNF580, MAP1S, CHEK1, ZNF32, YARS2, CDKAL1, LBR, MFF, UBE2M, GPS2, INO80C, SFMBT2, XPO1, HSF2, KRBA1, FAM184A, CIR1, POMT1, PCDH19, ZNF266, LRRC34, TOR1A, SGSM3, BAP1, C8orf34, DDB1, FYN, PSMG4, MED25, KIAA2013, SKI, AKAP17A, ARVCF, IRF2BP1, CDC40, THA

100%|█████████▉| 1601/1602 [00:23<00:00, 69.50it/s]


### Output file: 
`$outprefix.calibrated_distance.csv`

This file stores the calibrated distance and proximity for every pair of input proteins that have annotation in Gene Ontology (GO). Below are specific annotation for each column:
- **geneA:** name of gene A
- **geneB:** name of gene B
- **C:** the number of proteins in the smallest GO cellular component to which both are annotated
- **D:** protein-protein distance calibrated from GO
- **log10D:** $\log_{10} D$
- **P:** protein-protein proximity, $-\log_{10} D$

In [7]:
# calibrated_distance = pd.read_csv('./Examples/output/test.calibrated_distance.csv')
# calibrated_distance.head(2)

,geneA,geneB,C,log10D,D,P
0,SKI,SALL2,86,1.88471,76.684909,-1.88471
1,SKI,EID1,6356,3.84217,6952.964855,-3.84217


## Step 2: Build random forest to predict protein-protein proximity from data embeddings

Each random forest regressor in the original MuSIC study was trained with ~1M samples consisted of 2060 input features, requiring ~1 day and >100 Gb memory with 24 threads. As a demo, code below will generate two sets of image embeddings (MuSIC had six) and create 1000 samples for training.

Detailed documentation available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map#step-2-build-random-forest-to-predict-protein-protein-proximity-from-data-embeddings).

In [5]:
%%bash
python random_forest_samples.py \
--outprefix ./Examples/output/test \
--protein_file ./Examples/MuSIC_proteins.txt \
--emd_files ./Examples/IF_image_embedding.csv ./Examples/APMS_embedding.MuSIC.csv \
--emd_label IF_emd APMS_emd \
--num_set 2 auto \
--n_samples 1000 

Formatting IF_emd into 2 training sets using file ./Examples/IF_image_embedding.csv ...
Number of embeddings per protein ranges from 2 to 6.
Format IF_emd into 2 training sets...
Start calculating pairwise protein similarity for IF_emd with training set 1 ...
Start calculating pairwise protein similarity for IF_emd with training set 2 ...
Assess distinctness among different training sets generated (Jaccard)...


Formatting APMS_emd into auto training sets using file ./Examples/APMS_embedding.MuSIC.csv ...
Number of embeddings per protein ranges from 1 to 1.
Format APMS_emd into 1 training sets...
Start calculating pairwise protein similarity for APMS_emd with training set 1 ...


Split protein pairs into training and testing sets for 5-fold cross validation...
Sampled 890 training samples for fold 1
Sampled 900 training samples for fold 2
Sampled 880 training samples for fold 3
Sampled 900 training samples for fold 4
Sampled 880 training samples for fold 5

Start generating protein pai

In [8]:
%%bash
for ((fold = 1; fold <= 5; fold++))
do
    for ((IF_set = 1; IF_set <= 2; IF_set++))
    do
        python run_random_forest.py \
        --outprefix ./Examples/output/test \
        --fold $fold \
        --emd_label IF_emd APMS_emd \
        --train_set $IF_set 1 \
        --n_jobs 60;
    done
done

Trained model will be saved in ./Examples/output/test_trained_models/IF_emd_1_APMS_emd_1.RF_maxDep_30_nEst_1000.fold_1.pkl
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=60,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
... loaded training data
Start training random forest regressor on 890 samples with 2060 total features...
... finished training
... saved trained random forest regressor

Start predicting test data...
... loaded 36181 test samples
... finished predicting test samples
Pearson r: 0.18779669051059025

Start predicting gene pairs without specific GO annotations...

=== finished! ===
Trained model will be saved in ./Examples/output/test_trained_models/IF_emd_2_APMS_emd_1.RF_maxDep_30_nEst_100

/cellar/users/y8qin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_t

In [10]:
%%bash
python random_forest_output.py --outprefix ./Examples/output/test

=== finished! ===


### Output file: 
`$outprefix_predicted_proximity.txt`

This file stores the predicted protein-protein proximity for all pairs of the given protein.
- First column: name of gene A
- Second column: name of gene B
- Third column: predicted proximity between gene A and gene B

In [12]:
# predicted_proximity = pd.read_table('./Examples/output/test_predicted_proximity.txt', header=None)
# predicted_proximity.head(2)

,0,1,2
0,SKI,SALL2,0.372388
1,SKI,HSD17B6,0.306222


## Step 3: Analyze proximity data to identify protein communities at progressive resolutions

Please note this step takes ~7 hours.

Detailed documentation available [here](https://github.com/idekerlab/MuSIC/wiki/A-Step-By-Step-Guide-to-Building-a-MuSIC-Map#step-3-analyze-proximity-data-to-identify-protein-communities-at-progressive-resolutions).

In [13]:
%%bash
# Results from step 3 are only for demo, well-trained data are provided to reproduce MuSIC in step 4.
cp ./Examples/MuSIC_predicted_proximity.txt ./Examples/output/test_predicted_proximity.txt
cp ./Examples/MuSIC_avgPred_ytrue.csv ./Examples/output/test_avgPred_ytrue.csv

In [20]:
%%bash
python community_detection.py \
--outprefix ./Examples/output/test \
--path_to_clixo /cellar/users/y8qin/Modules/CliXO \
--clixo_i ./Examples/output/test_predicted_proximity.txt \
--clixo_a 0.01 --clixo_b 0.5 --clixo_m 0.008 --clixo_z 0.05 --min_diff 2 \
--path_to_alignOntology /cellar/users/y8qin/Modules/alignOntology-master \
--predict_nm_size --keep_all_files

# Note that CliXO can take ~7 hours for MuSIC. It's recommended to run the below command line in background
bash ./Examples/output/test.sh

Process is terminated.


### Output file: 
`$outprefix.louvain.ddot`

This file stores the hierarchical relationship among systems and genes:
- First column: the parent system 
- Second column: the child system or gene
- Third column: property of child in the second column
    - default: child is a system
    - gene: child is a gene

In [21]:
# music_hierarchy = pd.read_table('./Examples/output/test.louvain.ddot', header=None)
# music_hierarchy.head(2)

,0,1,2
0,739,714,default
1,739,MRPL46,gene


### Output file: 
`$outprefix.louvain.termStats`

This file stores the specific protein assignment for each identified system. First column is the unique identifier for each system. Other columns are annotated below:
- **Number_of_proteins:** total number of proteins belonging to the system
- **Proteins:** comma separated list of proteins belonging to the system
- **median_recal_nm:** median of predicted distance, in nm, among all pairs of proteins in the system
- **Estimated_size_in_nm:** predicted size, in nm, of the system

In [24]:
# music_system = pd.read_table('./Examples/output/test.louvain.termStats', index_col=0)
# music_system.head(2)

,Number_of_proteins,Proteins,median_recal_nm,Estimated_size_in_nm
0,,,,
739,6,"MRPL46,MRPS14,MRPL19,RPL27A,RPL38,MRPL20,",18.152899,32.504508
714,4,"MRPL19,RPL27A,RPL38,MRPL20,",3.059966,5.479163
